### Importing Libraries

In [2]:
import pandas as pd
import numpy as np
import texthero as hero
import re
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
import nltk
from nltk.stem import WordNetLemmatizer, SnowballStemmer
import spacy
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from gensim.models.phrases import Phrases, Phraser
from gingerit.gingerit import GingerIt

import emoji
import regex

import multiprocessing
import docx2txt

from gensim.models import Word2Vec

### Reading Data and basic Data Wrangling

In [62]:
df = pd.read_excel("BT4222_Combined_dataset_Version 2.xlsx")

In [3]:
raw_data = pd.read_csv("All Features on Raw Data Part 1.csv")

In [4]:
raw_data = raw_data.drop(columns=["Reddit","Twitter","Typology","Kaggle","Afinn Score","Polarity","Subjectivity","Length"])

In [5]:
raw_data

,Tag,Text,num_noun,num_adj,num_prep,num_det,num_pron,num_verb,num_adverb,num_interject,...,conjunction,pronoun,preposition,nominalization,pronoun.1,interrogative,article,subordination,conjunction.1,preposition.1
0,INFP,Meme<3,1,0,0,0,0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,ENFJ,"MemeIncorrect Quote? Not so sure. Just me, try...",6,1,1,1,2,2,2,0,...,0.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,INTP,MemeENFP Avatar,2,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,ENTP,MemeFour Distinct Flavors of NT,4,0,1,0,0,0,0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,INTJ,StereotypesINFP 🦋,2,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212133,ISFP,httpswww.youtube.comwatch?v=t8edHB_h908|||IxFP...,191,74,93,69,95,185,64,1,...,26.0,129.0,93.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0
212134,ENFP,'So...if this thread already exists someplace ...,314,91,116,91,201,329,112,0,...,75.0,251.0,140.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0
212135,INTP,'So many questions when i do these things. I ...,256,69,88,80,125,210,61,1,...,32.0,155.0,96.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0
212136,INFP,'I am very conflicted right now when it comes ...,323,113,218,137,253,398,183,3,...,63.0,300.0,216.0,13.0,0.0,0.0,0.0,0.0,0.0,0.0


In [163]:
df_mbti = df.copy()

In [164]:
df_mbti

,Tag,Text,Reddit,Twitter,Typology,Kaggle
0,INFP,Meme<3,1,0,0,0
1,ENFJ,"MemeIncorrect Quote? Not so sure. Just me, try...",1,0,0,0
2,INTP,MemeENFP Avatar,1,0,0,0
3,ENTP,MemeFour Distinct Flavors of NT,1,0,0,0
4,INTJ,StereotypesINFP 🦋,1,0,0,0
...,...,...,...,...,...,...
214114,ISFP,https://www.youtube.com/watch?v=t8edHB_h908|||...,0,0,0,1
214115,ENFP,'So...if this thread already exists someplace ...,0,0,0,1
214116,INTP,'So many questions when i do these things. I ...,0,0,0,1
214117,INFP,'I am very conflicted right now when it comes ...,0,0,0,1


In [165]:
df_mbti['Tag'] = df_mbti['Tag'].str.upper()

In [166]:
titles = ["MBTI", "ISTJ", "ISTP", "ISFJ", "ISFP", "INFJ", "INFP", "INTJ", "INTP", "ESTP", "ESTJ","ESFP", "ESFJ", "ENFP", "ENFJ", "ENTP", "ENTJ"]
df_mbti = df_mbti[df_mbti.Tag.isin(titles)]

In [167]:
df_mbti = df_mbti.dropna().drop_duplicates()

In [6]:
raw_data = raw_data.dropna().drop_duplicates()

In [150]:
emoticons_.extend(list(emoji.UNICODE_EMOJI['en'].keys()))

In [7]:
def emoticons_count(line):
    count = sum(item in i for i in line.split() for item in emoticons_)
    return count

In [170]:
df_mbti['Text'] = df_mbti['Text'].apply(lambda x: " ".join(x for x in str(x).split()))


In [171]:
df_mbti['Emoticons count'] = df_mbti['Text'].apply(lambda x: emoticons_count(x))

In [172]:
df_mbti

,Tag,Text,Reddit,Twitter,Typology,Kaggle,Emoticons count
0,INFP,Meme<3,1,0,0,0,1
1,ENFJ,"MemeIncorrect Quote? Not so sure. Just me, try...",1,0,0,0,1
2,INTP,MemeENFP Avatar,1,0,0,0,0
3,ENTP,MemeFour Distinct Flavors of NT,1,0,0,0,0
4,INTJ,StereotypesINFP 🦋,1,0,0,0,1
...,...,...,...,...,...,...,...
214114,ISFP,https://www.youtube.com/watch?v=t8edHB_h908|||...,0,0,0,1,13
214115,ENFP,'So...if this thread already exists someplace ...,0,0,0,1,22
214116,INTP,'So many questions when i do these things. I w...,0,0,0,1,2
214117,INFP,'I am very conflicted right now when it comes ...,0,0,0,1,1


#### Keeping only English words

In [173]:
df_mbti['Text'] =df_mbti['Text'].astype(str).apply(lambda x: ''.join([i if ord(i) < 128 else ' ' for i in x]))

In [8]:
raw_data['Text'] =raw_data['Text'].astype(str).apply(lambda x: ''.join([i if ord(i) < 128 else ' ' for i in x]))

In [174]:
df_mbti

,Tag,Text,Reddit,Twitter,Typology,Kaggle,Emoticons count
0,INFP,Meme<3,1,0,0,0,1
1,ENFJ,"MemeIncorrect Quote? Not so sure. Just me, try...",1,0,0,0,1
2,INTP,MemeENFP Avatar,1,0,0,0,0
3,ENTP,MemeFour Distinct Flavors of NT,1,0,0,0,0
4,INTJ,StereotypesINFP,1,0,0,0,1
...,...,...,...,...,...,...,...
214114,ISFP,https://www.youtube.com/watch?v=t8edHB_h908|||...,0,0,0,1,13
214115,ENFP,'So...if this thread already exists someplace ...,0,0,0,1,22
214116,INTP,'So many questions when i do these things. I w...,0,0,0,1,2
214117,INFP,'I am very conflicted right now when it comes ...,0,0,0,1,1


### Preprocessing

#### Expanding contractions


In [13]:
contractions_re=re.compile('(%s)' % '|'.join(contractions_dict.keys()))

# Function for expanding contractions
def expand_contractions(text,contractions_dict=contractions_dict):
    def replace(match):
        return contractions_dict[match.group(0)]
    return contractions_re.sub(replace, text)

In [176]:
df_mbti['Text'] =df_mbti['Text'].astype(str).apply(lambda x: expand_contractions(x))

In [14]:
raw_data['Text'] =raw_data['Text'].astype(str).apply(lambda x: expand_contractions(x))

#### Cleaning with texthero

In [15]:
custom_pipeline = [hero.preprocessing.fillna,
                   hero.preprocessing.lowercase,
                   hero.preprocessing.remove_whitespace,
                   hero.preprocessing.remove_urls,
                   hero.preprocessing.remove_diacritics,
                   hero.preprocessing.remove_urls,
                   hero.preprocessing.remove_html_tags
                   ]




In [178]:
df_mbti['Text'] = df_mbti['Text'].pipe(hero.clean, custom_pipeline)

In [16]:
raw_data['Text'] = raw_data['Text'].pipe(hero.clean, custom_pipeline)

In [179]:
df_mbti

,Tag,Text,Reddit,Twitter,Typology,Kaggle,Emoticons count
0,INFP,meme<3,1,0,0,0,1
1,ENFJ,"memeincorrect quote? not so sure. just me, try...",1,0,0,0,1
2,INTP,memeenfp avatar,1,0,0,0,0
3,ENTP,memefour distinct flavors of nt,1,0,0,0,0
4,INTJ,stereotypesinfp,1,0,0,0,1
...,...,...,...,...,...,...,...
214114,ISFP,just because i always think of cats as fi do...,0,0,0,1,13
214115,ENFP,'so...if this thread already exists someplace ...,0,0,0,1,22
214116,INTP,'so many questions when i do these things. i w...,0,0,0,1,2
214117,INFP,'i am very conflicted right now when it comes ...,0,0,0,1,1


#### Retaining Emoticons

In [17]:
emoticons = {
    ":)": "absmile",
    "<3": "bcheart",
    "(:": "cdsmile",
    "(-:": "desmile",
    ":-)": "efsmile",
    "(=": "fgsmile",
    "=)": "ghsmile",
    ":p": "hjwink",
    ":P": "ijwink",
    ":D": "jksmile",
    "xD": "klsmile",
    "XD": "lmsmile",
    ":/": "mndisap",
    "/:": "nodisap",
    ":|": "opdisap",
    ":]": "pqsmile",
    "8)": "qrsmile",
    "=D": "rssmile",
    "8D": "stsmile",
    ";D": "tuwink",
    ";)": "uvwink",
    "):": "vwsad",
    ");": "wxsad",
    ":-(": "xysad",
    ":[": "yzsad",
    ":')": "zalaugh",
    ":'(": "abecry",
    ":x": "cbdkiss",
    ":o": "dfgsurprise",
    ":0": "xyshock",
    ":O": "xysurprise",
    "D;": "xydisgust",
    "DX": "xyhorror",
    "D8": "abhorror"

}

emoticons_inverse = {v: k for k, v in emoticons.items()}

punctuation = ",./<>?;':\"[]\\{}|`~!@#$%^&*()_+-="

emoticons_ = list(emoticons.keys())

In [18]:
lines_clean = []

for line in raw_data['Text']:
    #Replace emoticons with non-punctuation
    for emote, rpl in emoticons.items():
        line = line.replace(emote, " "+rpl+" ")
        
    line = line.replace('\d+', '')
    #Remove punctuation
    for char in line:
        if char in punctuation:
            line = line.replace(char, "")

    #Revert emoticons
    for emote, rpl in emoticons_inverse.items():
        line = line.replace(emote, rpl)

    lines_clean.append(line)

raw_data['Text'] = lines_clean


In [ ]:
lines_clean = []

for line in df_mbti['Text']:
    #Replace emoticons with non-punctuation
    for emote, rpl in emoticons.items():
        line = line.replace(emote, " "+rpl+" ")
        
    line = line.replace('\d+', '')
    #Remove punctuation
    for char in line:
        if char in punctuation:
            line = line.replace(char, "")

    #Revert emoticons
    for emote, rpl in emoticons_inverse.items():
        line = line.replace(emote, rpl)

    lines_clean.append(line)

df_mbti['Text'] = lines_clean


In [181]:
df_mbti['Text'] = hero.remove_whitespace(df_mbti['Text'])

#### Removing Stopwords

In [19]:
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):


    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # delete stopwors from text
    return text


#df_mbti['Text'] =df_mbti['Text'].astype(str).apply(clean_text)
raw_data['Text'] =raw_data['Text'].astype(str).apply(clean_text)

#### Dropping empty rows and duplicated rows

In [183]:
df_mbti = df_mbti.dropna().drop_duplicates()

In [20]:
raw_data = raw_data.dropna().drop_duplicates()

#### Spelling Harmonisation

In [21]:
def replace_all(text, mydict):
    for gb, us in mydict.items():
        text = text.replace(us, gb)
    return text

In [187]:
#harmonising spelling
df_mbti['Text'] = df_mbti['Text'].apply(lambda x: replace_all(x, us2gb))

# Run the cell with us2gb all the way at the bottom of the notebook!!!

In [22]:
raw_data['Text'] = raw_data['Text'].apply(lambda x: replace_all(x, us2gb))

#### Stemming and processing using Gensim's library

In [188]:
def lemmatize_stemming(text):
    stemmer = SnowballStemmer("english")
    return stemmer.stem(text)
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [189]:
df_mbti['Text_stemmed'] = df_mbti['Text'].map(preprocess)
df_mbti['Text_stemmed'] = [','.join(map(str, l)).replace(",", " ") for l in df_mbti['Text_stemmed']]


#### Removing lines with less than 10 characters or less than or equal to 2 words

In [190]:
df_mbti = df_mbti[~df_mbti['Text'].str.split().str.len().lt(3)]
# df_mbti = df_mbti[df_mbti['Cleaned_text'].apply(len)>10]

In [23]:
raw_data = raw_data[~raw_data['Text'].str.split().str.len().lt(3)]

#### Can use this for obtaining Noun Chunks but it takes too Long to run. I have used Bigrams instead. Similar purpose

In [25]:
nlp = spacy.load('en_core_web_sm')

In [26]:
def nc_gen(line):

    chunk = []
    for word in nlp(line).noun_chunks:
            chunk.append(word)
    return chunk
        
        
# df_mbti['Noun_chunks'] = df_mbti['Cleaned_Text'].apply(lambda x: nc_gen(x)) 
# too slow

### Vectorising. 

#### Can play around with the type and parameters

In [191]:
#vect = TfidfVectorizer()
vect = CountVectorizer(max_df=0.7, max_features=5000)
#can change the max_features, max_df
dtm = pd.DataFrame(vect.fit_transform(df_mbti['Text']).toarray(), columns=vect.get_feature_names())


In [192]:
dtm

,00100000,10,100,1000,11,12,13,14,15,16,...,youre,youri,youth,youtube,youve,yup,zero,zodiac,zombie,zone
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127578,0,0,0,0,0,0,0,0,0,0,...,1,0,0,1,0,0,0,0,0,0
127579,0,0,0,0,0,0,0,0,0,0,...,5,0,0,0,0,0,0,0,0,0
127580,0,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
127581,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [29]:
df_mbti = df_mbti.reset_index().drop(columns = ["index"])

In [24]:
raw_data = raw_data.reset_index().drop(columns = ["index"])

In [193]:
print(vect.get_feature_names()[-50:])

['wrap', 'wrapped', 'write', 'writer', 'writers', 'writing', 'written', 'wrong', 'wrote', 'wtf', 'xd', 'xdi', 'xnfp', 'xntp', 'xstj', 'ya', 'yall', 'yang', 'yay', 'yea', 'yeah', 'year', 'years', 'yell', 'yelling', 'yellow', 'yep', 'yes', 'yesterday', 'yet', 'yeu', 'yo', 'yoga', 'york', 'youd', 'youi', 'youll', 'young', 'younger', 'youngest', 'youre', 'youri', 'youth', 'youtube', 'youve', 'yup', 'zero', 'zodiac', 'zombie', 'zone']


In [32]:
len(df_mbti)

127577

In [194]:
df_mbti.iloc[120000]['Text']

'understand x existshappens think going make story make sure tell children think religioni actually find estps one easiest types identify theyre much better repeating working process long keep profiting one odd jobsthis thing youth think still hit wall time time learned push magic pull sudden makes sudden immune toare currently committed allowing one person plow foreseeable futuregetting ball rolling projects probably opportunistic nts xntjs plan plan something satisfied fail intps actually likewell difference estp entp salesman estp memorize technical jargon repeat client whereas entp actually understand technicali business last three years decided close primarily due retarded supplier hurt business point could recover none ihello percy friends absent forums months mainly lost interest mbti little although automatically trying type everyone meet nicethe best way describe beliefs agnostic atheist cannot definitively prove god exist cannot definitively prove teacup orbiting around jupit

In [195]:
def summarize(message_id):

    message_text = df_mbti.loc[message_id, 'Text']
    print(message_text)
    print(df_mbti.loc[message_id, 'Tag'])

    
    # create a list of all unique words in the review (minus stop words) using CountVectorizer
    unique_words = vect.get_feature_names()
    
    # create a dictionary of words and their TF-IDF scores
    word_scores = {}
    for word in unique_words:
        word_scores[word] = dtm.iloc[message_id, unique_words.index(word)]
        
    
    # print words with the top 5 TF-IDF scores
    print('\n'+'TOP SCORING WORDS:')
    top_scores = sorted(word_scores.items(), key=lambda x: x[1], reverse=True)[0:3]
    for word, score in top_scores:
        print(word)
    
    # print 5 random words (for comparison)
    print('\n' + 'RANDOM WORDS:')
    random_words = np.random.choice(list(word_scores.keys()), size=3, replace=False)
    for word in random_words:
        print(word)
    


In [196]:
summarize(120000)

two former illini starting lineup miami heat tonight nunn leonard
ESTP

TOP SCORING WORDS:
people
think
time

RANDOM WORDS:
react
attitudes
figured


### Bigram

In [197]:
def letters(s):
    return re.sub('[^a-zA-Z]+', " ", s)

In [198]:
df_mbti['Cleaned_Text_No_Emoticon'] = df_mbti['Text'].apply(lambda x: letters(x))

In [199]:
gram = [row.split() for row in df_mbti.loc[:, 'Cleaned_Text_No_Emoticon']]
bigram_phrases = Phrases(gram, min_count=10, progress_per=10000)
trigram_phrases = Phrases(bigram_phrases[gram], min_count=1, progress_per=10000)
bigram = Phraser(bigram_phrases)
trigram = Phraser(trigram_phrases)
sentences_b = bigram[gram]
sentences_t = trigram[gram]
df_mbti["Bigram"] = sentences_b
df_mbti['Bigram'] = [','.join(map(str, l)).replace(",", " ") for l in df_mbti['Bigram']]

df_mbti["Trigram"] = sentences_t
df_mbti['Trigram'] = [','.join(map(str, l)).replace(",", " ") for l in df_mbti['Trigram']]






C:\Users\valli\anaconda3\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [200]:
df_mbti

,Tag,Text,Reddit,Twitter,Typology,Kaggle,Emoticons count,Text_stemmed,Cleaned_Text_No_Emoticon,Bigram,Trigram
1,ENFJ,memeincorrect quote sure trying keep peace family,1,0,0,0,1,memeincorrect quot sure tri peac famili,memeincorrect quote sure trying keep peace family,memeincorrect quote sure trying keep_peace family,memeincorrect quote sure trying keep_peace family
3,ENTP,memefour distinct flavors nt,1,0,0,0,0,memefour distinct flavor,memefour distinct flavors nt,memefour distinct flavors nt,memefour distinct flavors nt
6,ENTP,memethis definitely intp looks like,1,0,0,0,0,memethi definit intp look like,memethis definitely intp looks like,memethis definitely intp looks_like,memethis definitely intp looks_like
7,ISFJ,theory questionwhat type admire,1,0,0,0,0,theori questionwhat type admir,theory questionwhat type admire,theory_questionwhat type admire,theory_questionwhat type admire
8,ISTJ,memeistj x enfp real,1,0,0,0,0,memeistj enfp real,memeistj x enfp real,memeistj x enfp real,memeistj x enfp real
...,...,...,...,...,...,...,...,...,...,...,...
214114,ISFP,always think cats fi doms reason websites beco...,0,0,0,1,13,think cat dom reason websit nazi perci nerd le...,always think cats fi doms reason websites beco...,always think cats fi_doms reason websites beco...,always think cats fi_doms reason websites beco...
214115,ENFP,soif thread already exists someplace else heck...,0,0,0,1,22,soif thread exist someplac heck delet hereooop...,soif thread already exists someplace else heck...,soif thread already_exists someplace else heck...,soif thread already_exists someplace_else heck...
214116,INTP,many questions things would take purple pill p...,0,0,0,1,2,question thing purpl pill pick win lotteri num...,many questions things would take purple pill p...,many questions things would take purple pill p...,many questions things would take purple pill p...
214117,INFP,conflicted right comes wanting children honest...,0,0,0,1,1,conflict right come want children honest mater...,conflicted right comes wanting children honest...,conflicted right comes wanting children honest...,conflicted right comes wanting children honest...


In [201]:
df_mbti = df_mbti[["Tag", "Text", "Cleaned_Text_No_Emoticon", "Text_stemmed", "Bigram", "Trigram", "Emoticons count", "Reddit", "Twitter", "Typology", "Kaggle"]]
df_mbti.columns = ["Tag", "Cleaned_Text", "Cleaned_Text_No_Emoticon", "Text_stemmed", "Bigram", "Trigram", "Emoticons_count", "Reddit", "Twitter", "Typology", "Kaggle"]


In [202]:
df_mbti

,Tag,Cleaned_Text,Cleaned_Text_No_Emoticon,Text_stemmed,Bigram,Trigram,Emoticons_count,Reddit,Twitter,Typology,Kaggle
1,ENFJ,memeincorrect quote sure trying keep peace family,memeincorrect quote sure trying keep peace family,memeincorrect quot sure tri peac famili,memeincorrect quote sure trying keep_peace family,memeincorrect quote sure trying keep_peace family,1,1,0,0,0
3,ENTP,memefour distinct flavors nt,memefour distinct flavors nt,memefour distinct flavor,memefour distinct flavors nt,memefour distinct flavors nt,0,1,0,0,0
6,ENTP,memethis definitely intp looks like,memethis definitely intp looks like,memethi definit intp look like,memethis definitely intp looks_like,memethis definitely intp looks_like,0,1,0,0,0
7,ISFJ,theory questionwhat type admire,theory questionwhat type admire,theori questionwhat type admir,theory_questionwhat type admire,theory_questionwhat type admire,0,1,0,0,0
8,ISTJ,memeistj x enfp real,memeistj x enfp real,memeistj enfp real,memeistj x enfp real,memeistj x enfp real,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
214114,ISFP,always think cats fi doms reason websites beco...,always think cats fi doms reason websites beco...,think cat dom reason websit nazi perci nerd le...,always think cats fi_doms reason websites beco...,always think cats fi_doms reason websites beco...,13,0,0,0,1
214115,ENFP,soif thread already exists someplace else heck...,soif thread already exists someplace else heck...,soif thread exist someplac heck delet hereooop...,soif thread already_exists someplace else heck...,soif thread already_exists someplace_else heck...,22,0,0,0,1
214116,INTP,many questions things would take purple pill p...,many questions things would take purple pill p...,question thing purpl pill pick win lotteri num...,many questions things would take purple pill p...,many questions things would take purple pill p...,2,0,0,0,1
214117,INFP,conflicted right comes wanting children honest...,conflicted right comes wanting children honest...,conflict right come want children honest mater...,conflicted right comes wanting children honest...,conflicted right comes wanting children honest...,1,0,0,0,1


In [25]:
raw_data = raw_data.rename(columns={"Text": "Cleaned_Text"})

In [26]:
raw_data.to_csv("raw_data.csv")

In [27]:
raw_data

,Tag,Cleaned_Text,num_noun,num_adj,num_prep,num_det,num_pron,num_verb,num_adverb,num_interject,...,conjunction,pronoun,preposition,nominalization,pronoun.1,interrogative,article,subordination,conjunction.1,preposition.1
0,ENFJ,memeincorrect quote sure trying keep peace family,6,1,1,1,2,2,2,0,...,0.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,ENTP,memefour distinct flavors nt,4,0,1,0,0,0,0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,ENTP,memethis definitely intp looks like,2,0,1,0,1,2,1,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,ISFJ,theory questionwhat type admire,3,0,0,1,1,2,0,0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,ISTJ,memeistj x enfp real,6,1,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151891,ISFP,always think cats fi doms reason websites beco...,191,74,93,69,95,185,64,1,...,26.0,129.0,93.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0
151892,ENFP,soif thread already exists someplace else heck...,314,91,116,91,201,329,112,0,...,75.0,251.0,140.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0
151893,INTP,many questions things would take purple pill p...,256,69,88,80,125,210,61,1,...,32.0,155.0,96.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0
151894,INFP,conflicted right comes wanting children honest...,323,113,218,137,253,398,183,3,...,63.0,300.0,216.0,13.0,0.0,0.0,0.0,0.0,0.0,0.0


In [203]:
df_mbti.to_csv("MBTI.csv")

### Building corpus using word2vec

In [34]:
cores = multiprocessing.cpu_count()
w2v_model = Word2Vec(min_count=10,
                     window=3,
                     size=15,
                     sample=6e-5, 
                     alpha=0.05, 
                     min_alpha=0.0007, 
                     negative=20,
                     workers=cores-1)

In [35]:
w2v_model.build_vocab(sentences, progress_per=10000)

In [36]:
w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs=5, report_delay=1)


(6983770, 12552250)

In [ ]:
w2v_model.wv.most_similar(positive=["yes"])

In [11]:
contractions_dict = { "ain't": "are not","'s":" is","aren't": "are not",
                     "can't": "cannot","can't've": "cannot have",
                     "'cause": "because","could've": "could have","couldn't": "could not",
                     "couldn't've": "could not have", "didn't": "did not","doesn't": "does not",
                     "don't": "do not","hadn't": "had not","hadn't've": "had not have",
                     "hasn't": "has not","haven't": "have not","he'd": "he would",
                     "he'd've": "he would have","he'll": "he will", "he'll've": "he will have",
                     "how'd": "how did","how'd'y": "how do you","how'll": "how will",
                     "I'd": "I would", "I'd've": "I would have","I'll": "I will",
                     "I'll've": "I will have","I'm": "I am","I've": "I have", "isn't": "is not",
                     "it'd": "it would","it'd've": "it would have","it'll": "it will",
                     "it'll've": "it will have", "let's": "let us","ma'am": "madam",
                     "mayn't": "may not","might've": "might have","mightn't": "might not", 
                     "mightn't've": "might not have","must've": "must have","mustn't": "must not",
                     "mustn't've": "must not have", "needn't": "need not",
                     "needn't've": "need not have","o'clock": "of the clock","oughtn't": "ought not",
                     "oughtn't've": "ought not have","shan't": "shall not","sha'n't": "shall not",
                     "shan't've": "shall not have","she'd": "she would","she'd've": "she would have",
                     "she'll": "she will", "she'll've": "she will have","should've": "should have",
                     "shouldn't": "should not", "shouldn't've": "should not have","so've": "so have",
                     "that'd": "that would","that'd've": "that would have", "there'd": "there would",
                     "there'd've": "there would have", "they'd": "they would",
                     "they'd've": "they would have","they'll": "they will",
                     "they'll've": "they will have", "they're": "they are","they've": "they have",
                     "to've": "to have","wasn't": "was not","we'd": "we would",
                     "we'd've": "we would have","we'll": "we will","we'll've": "we will have",
                     "we're": "we are","we've": "we have", "weren't": "were not","what'll": "what will",
                     "what'll've": "what will have","what're": "what are", "what've": "what have",
                     "when've": "when have","where'd": "where did", "where've": "where have",
                     "who'll": "who will","who'll've": "who will have","who've": "who have",
                     "why've": "why have","will've": "will have","won't": "will not",
                     "won't've": "will not have", "would've": "would have","wouldn't": "would not",
                     "wouldn't've": "would not have","y'all": "you all", "y'all'd": "you all would",
                     "y'all'd've": "you all would have","y'all're": "you all are",
                     "y'all've": "you all have", "you'd": "you would","you'd've": "you would have",
                     "you'll": "you will","you'll've": "you will have", "you're": "you are",
                     "you've": "you have"}


In [10]:
us2gb = {'accessorize': 'accessorise', 'accessorized': 'accessorised', 'accessorizes': 'accessorises', 'accessorizing': 'accessorising',
 'acclimatization': 'acclimatisation','acclimatize': 'acclimatise','acclimatized': 'acclimatised','acclimatizes': 'acclimatises','acclimatizing': 'acclimatising',
 'accouterments': 'accoutrements','aerogram': 'aerogramme','aerograms': 'aerogrammes','aggrandizement': 'aggrandisement',
 'aging': 'ageing','agonize': 'agonise','agonized': 'agonised','agonizes': 'agonises','agonizing': 'agonising',
 'agonizingly': 'agonisingly','airplane': 'aeroplane','airplanes ': 'aeroplanes ','almanac': 'almanack','almanacs': 'almanacks','aluminum': 'aluminium',
 'amortizable': 'amortisable','amortization': 'amortisation','amortizations': 'amortisations','amortize': 'amortise',
 'amortized': 'amortised','amortizes': 'amortises','amortizing': 'amortising','amphitheater': 'amphitheatre',
 'amphitheaters': 'amphitheatres','analog': 'analogue','analogs': 'analogues','analyze': 'analyse','analyzed': 'analysed',
 'analyzes': 'analyses','analyzing': 'analysing','anemia': 'anaemia','anemic': 'anaemic','anesthesia': 'anaesthesia',
 'anesthetic': 'anaesthetic','anesthetics': 'anaesthetics','anesthetist': 'anaesthetist','anesthetists': 'anaesthetists','anesthetize': 'anaesthetize',
 'anesthetized': 'anaesthetized','anesthetizes': 'anaesthetizes','anesthetizing': 'anaesthetizing','anglicize': 'anglicise',
 'anglicized': 'anglicised','anglicizes': 'anglicises','anglicizing': 'anglicising','annualized': 'annualised',
 'antagonize': 'antagonise','antagonized': 'antagonised','antagonizes': 'antagonises','antagonizing': 'antagonising',
 'apologize': 'apologise','apologized': 'apologised','apologizes': 'apologises','apologizing': 'apologising',
 'appall': 'appal','appalls': 'appals','appetizer': 'appetiser','appetizers': 'appetisers','appetizing': 'appetising',
 'appetizingly': 'appetisingly','arbor': 'arbour','arbors': 'arbours','archeological': 'archaeological','archeologically': 'archaeologically',
 'archeologist': 'archaeologist','archeologists': 'archaeologists','archeology': 'archaeology','ardor': 'ardour',
 'armor': 'armour','armored': 'armoured','armorer': 'armourer','armorers': 'armourers','armories': 'armouries','armory': 'armoury',
 'artifact': 'artefact',
 'artifacts': 'artefacts',
 'authorize': 'authorise',
 'authorized': 'authorised',
 'authorizes': 'authorises',
 'authorizing': 'authorising',
 'ax': 'axe',
 'backpedaled': 'backpedalled',
 'backpedaling': 'backpedalling',
 'balk': 'baulk',
 'balked': 'baulked',
 'balking': 'baulking',
 'balks': 'baulks',
 'banister': 'bannister',
 'banisters': 'bannisters',
 'baptize': 'baptise',
 'baptized': 'baptised',
 'baptizes': 'baptises',
 'baptizing': 'baptising',
 'bastardize': 'bastardise',
 'bastardized': 'bastardised',
 'bastardizes': 'bastardises',
 'bastardizing': 'bastardising',
 'battleax': 'battleaxe',
 'bedeviled': 'bedevilled',
 'bedeviling': 'bedevilling',
 'behavior': 'behaviour',
 'behavioral': 'behavioural',
 'behaviorism': 'behaviourism',
 'behaviorist': 'behaviourist',
 'behaviorists': 'behaviourists',
 'behaviors': 'behaviours',
 'behoove': 'behove',
 'behooved': 'behoved',
 'behooves': 'behoves',
 'bejeweled': 'bejewelled',
 'belabor': 'belabour',
 'belabored': 'belaboured',
 'belaboring': 'belabouring',
 'belabors': 'belabours',
 'beveled': 'bevelled',
 'bevies': 'bevvies',
 'bevy': 'bevvy',
 'biased': 'biassed',
 'biasing': 'biassing',
 'binging': 'bingeing',
 'bougainvillea': 'bougainvillaea',
 'bougainvilleas': 'bougainvillaeas',
 'bowdlerize': 'bowdlerise',
 'bowdlerized': 'bowdlerised',
 'bowdlerizes': 'bowdlerises',
 'bowdlerizing': 'bowdlerising',
 'breathalyze': 'breathalyse',
 'breathalyzed': 'breathalysed',
 'breathalyzer': 'breathalyser',
 'breathalyzers': 'breathalysers',
 'breathalyzes': 'breathalyses',
 'breathalyzing': 'breathalysing',
 'brutalize': 'brutalise',
 'brutalized': 'brutalised',
 'brutalizes': 'brutalises',
 'brutalizing': 'brutalising',
 'busses': 'buses',
 'bussing': 'busing',
 'caliber': 'calibre',
 'calibers': 'calibres',
 'caliper': 'calliper',
 'calipers': 'callipers',
 'calisthenics': 'callisthenics',
 'canalize': 'canalise',
 'canalized': 'canalised',
 'canalizes': 'canalises',
 'canalizing': 'canalising',
 'cancelation': 'cancellation',
 'cancelations': 'cancellations',
 'canceled': 'cancelled',
 'canceling': 'cancelling',
 'candor': 'candour',
 'cannibalize': 'cannibalise',
 'cannibalized': 'cannibalised',
 'cannibalizes': 'cannibalises',
 'cannibalizing': 'cannibalising',
 'canonize': 'canonise',
 'canonized': 'canonised',
 'canonizes': 'canonises',
 'canonizing': 'canonising',
 'capitalize': 'capitalise',
 'capitalized': 'capitalised',
 'capitalizes': 'capitalises',
 'capitalizing': 'capitalising',
 'caramelize': 'caramelise',
 'caramelized': 'caramelised',
 'caramelizes': 'caramelises',
 'caramelizing': 'caramelising',
 'carbonize': 'carbonise',
 'carbonized': 'carbonised',
 'carbonizes': 'carbonises',
 'carbonizing': 'carbonising',
 'caroled': 'carolled',
 'caroling': 'carolling',
 'catalog': 'catalogue',
 'cataloged': 'catalogued',
 'cataloging': 'cataloguing',
 'catalogs': 'catalogues',
 'catalyze': 'catalyse',
 'catalyzed': 'catalysed',
 'catalyzes': 'catalyses',
 'catalyzing': 'catalysing',
 'categorize': 'categorise',
 'categorized': 'categorised',
 'categorizes': 'categorises',
 'categorizing': 'categorising',
 'cauterize': 'cauterise',
 'cauterized': 'cauterised',
 'cauterizes': 'cauterises',
 'cauterizing': 'cauterising',
 'caviled': 'cavilled',
 'caviling': 'cavilling',
 'center': 'centre',
 'centered': 'centred',
 'centerfold': 'centrefold',
 'centerfolds': 'centrefolds',
 'centerpiece': 'centrepiece',
 'centerpieces': 'centrepieces',
 'centers': 'centres',
 'centigram': 'centigramme',
 'centigrams': 'centigrammes',
 'centiliter': 'centilitre',
 'centiliters': 'centilitres',
 'centimeter': 'centimetre',
 'centimeters': 'centimetres',
 'centralize': 'centralise',
 'centralized': 'centralised',
 'centralizes': 'centralises',
 'centralizing': 'centralising',
 'cesarean': 'caesarean',
 'cesareans': 'caesareans',
 'channeled': 'channelled',
 'channeling': 'channelling',
 'characterize': 'characterise',
 'characterized': 'characterised',
 'characterizes': 'characterises',
 'characterizing': 'characterising',
 'check': 'cheque',
 'checkbook': 'chequebook',
 'checkbooks': 'chequebooks',
 'checkered': 'chequered',
 'checks': 'cheques',
 'chili': 'chilli',
 'chimera': 'chimaera',
 'chimeras': 'chimaeras',
 'chiseled': 'chiselled',
 'chiseling': 'chiselling',
 'cipher': 'cypher',
 'ciphers': 'cyphers',
 'circularize': 'circularise',
 'circularized': 'circularised',
 'circularizes': 'circularises',
 'circularizing': 'circularising',
 'civilize': 'civilise',
 'civilized': 'civilised',
 'civilizes': 'civilises',
 'civilizing': 'civilising',
 'clamor': 'clamour',
 'clamored': 'clamoured',
 'clamoring': 'clamouring',
 'clamors': 'clamours',
 'clangor': 'clangour',
 'clarinetist': 'clarinettist',
 'clarinetists': 'clarinettists',
 'collectivize': 'collectivise',
 'collectivized': 'collectivised',
 'collectivizes': 'collectivises',
 'collectivizing': 'collectivising',
 'colonization': 'colonisation',
 'colonize': 'colonise',
 'colonized': 'colonised',
 'colonizer': 'coloniser',
 'colonizers': 'colonisers',
 'colonizes': 'colonises',
 'colonizing': 'colonising',
 'color': 'colour',
 'colorant': 'colourant',
 'colorants': 'colourants',
 'colored': 'coloured',
 'coloreds': 'coloureds',
 'colorful': 'colourful',
 'colorfully': 'colourfully',
 'coloring': 'colouring',
 'colorize': 'colourize',
 'colorized': 'colourized',
 'colorizes': 'colourizes',
 'colorizing': 'colourizing',
 'colorless': 'colourless',
 'colors': 'colours',
 'commercialize': 'commercialise',
 'commercialized': 'commercialised',
 'commercializes': 'commercialises',
 'commercializing': 'commercialising',
 'compartmentalize': 'compartmentalise',
 'compartmentalized': 'compartmentalised',
 'compartmentalizes': 'compartmentalises',
 'compartmentalizing': 'compartmentalising',
 'computerize': 'computerise',
 'computerized': 'computerised',
 'computerizes': 'computerises',
 'computerizing': 'computerising',
 'conceptualize': 'conceptualise',
 'conceptualized': 'conceptualised',
 'conceptualizes': 'conceptualises',
 'conceptualizing': 'conceptualising',
 'connection': 'connexion',
 'connections': 'connexions',
 'contextualize': 'contextualise',
 'contextualized': 'contextualised',
 'contextualizes': 'contextualises',
 'contextualizing': 'contextualising',
 'councilor': 'councillor',
 'councilors': 'councillors',
 'counseled': 'counselled',
 'counseling': 'counselling',
 'counselor': 'counsellor',
 'counselors': 'counsellors',
 'cozier': 'cosier',
 'cozies': 'cosies',
 'coziest': 'cosiest',
 'cozily': 'cosily',
 'coziness': 'cosiness',
 'cozy': 'cosy',
 'crenelated': 'crenellated',
 'criminalize': 'criminalise',
 'criminalized': 'criminalised',
 'criminalizes': 'criminalises',
 'criminalizing': 'criminalising',
 'criticize': 'criticise',
 'criticized': 'criticised',
 'criticizes': 'criticises',
 'criticizing': 'criticising',
 'crueler': 'crueller',
 'cruelest': 'cruellest',
 'crystallization': 'crystallisation',
 'crystallize': 'crystallise',
 'crystallized': 'crystallised',
 'crystallizes': 'crystallises',
 'crystallizing': 'crystallising',
 'cudgeled': 'cudgelled',
 'cudgeling': 'cudgelling',
 'customize': 'customise',
 'customized': 'customised',
 'customizes': 'customises',
 'customizing': 'customising',
 'decentralization': 'decentralisation',
 'decentralize': 'decentralise',
 'decentralized': 'decentralised',
 'decentralizes': 'decentralises',
 'decentralizing': 'decentralising',
 'decriminalization': 'decriminalisation',
 'decriminalize': 'decriminalise',
 'decriminalized': 'decriminalised',
 'decriminalizes': 'decriminalises',
 'decriminalizing': 'decriminalising',
 'defense': 'defence',
 'defenseless': 'defenceless',
 'defenses': 'defences',
 'dehumanization': 'dehumanisation',
 'dehumanize': 'dehumanise',
 'dehumanized': 'dehumanised',
 'dehumanizes': 'dehumanises',
 'dehumanizing': 'dehumanising',
 'demeanor': 'demeanour',
 'demilitarization': 'demilitarisation',
 'demilitarize': 'demilitarise',
 'demilitarized': 'demilitarised',
 'demilitarizes': 'demilitarises',
 'demilitarizing': 'demilitarising',
 'demobilization': 'demobilisation',
 'demobilize': 'demobilise',
 'demobilized': 'demobilised',
 'demobilizes': 'demobilises',
 'demobilizing': 'demobilising',
 'democratization': 'democratisation',
 'democratize': 'democratise',
 'democratized': 'democratised',
 'democratizes': 'democratises',
 'democratizing': 'democratising',
 'demonize': 'demonise',
 'demonized': 'demonised',
 'demonizes': 'demonises',
 'demonizing': 'demonising',
 'demoralization': 'demoralisation',
 'demoralize': 'demoralise',
 'demoralized': 'demoralised',
 'demoralizes': 'demoralises',
 'demoralizing': 'demoralising',
 'denationalization': 'denationalisation',
 'denationalize': 'denationalise',
 'denationalized': 'denationalised',
 'denationalizes': 'denationalises',
 'denationalizing': 'denationalising',
 'deodorize': 'deodorise',
 'deodorized': 'deodorised',
 'deodorizes': 'deodorises',
 'deodorizing': 'deodorising',
 'depersonalize': 'depersonalise',
 'depersonalized': 'depersonalised',
 'depersonalizes': 'depersonalises',
 'depersonalizing': 'depersonalising',
 'deputize': 'deputise',
 'deputized': 'deputised',
 'deputizes': 'deputises',
 'deputizing': 'deputising',
 'desensitization': 'desensitisation',
 'desensitize': 'desensitise',
 'desensitized': 'desensitised',
 'desensitizes': 'desensitises',
 'desensitizing': 'desensitising',
 'destabilization': 'destabilisation',
 'destabilize': 'destabilise',
 'destabilized': 'destabilised',
 'destabilizes': 'destabilises',
 'destabilizing': 'destabilising',
 'dialed': 'dialled',
 'dialing': 'dialling',
 'dialog': 'dialogue',
 'dialogs': 'dialogues',
 'diarrhea': 'diarrhoea',
 'digitize': 'digitise',
 'digitized': 'digitised',
 'digitizes': 'digitises',
 'digitizing': 'digitising',
 'discolor': 'discolour',
 'discolored': 'discoloured',
 'discoloring': 'discolouring',
 'discolors': 'discolours',
 'disemboweled': 'disembowelled',
 'disemboweling': 'disembowelling',
 'disfavor': 'disfavour',
 'disheveled': 'dishevelled',
 'passivizes': 'passivises',
 'passivizing': 'passivising',
 'pasteurization': 'pasteurisation',
 'pasteurize': 'pasteurise',
 'pasteurized': 'pasteurised',
 'pasteurizes': 'pasteurises',
 'pasteurizing': 'pasteurising',
 'patronize': 'patronise',
 'patronized': 'patronised',
 'patronizes': 'patronises',
 'patronizing': 'patronising',
 'patronizingly': 'patronisingly',
 'pedaled': 'pedalled',
 'pedaling': 'pedalling',
 'pederast': 'paederast',
 'pederasts': 'paederasts',
 'pedestrianization': 'pedestrianisation',
 'pedestrianize': 'pedestrianise',
 'pedestrianized': 'pedestrianised',
 'pedestrianizes': 'pedestrianises',
 'pedestrianizing': 'pedestrianising',
 'pediatric': 'paediatric',
 'pediatrician': 'paediatrician',
 'pediatricians': 'paediatricians',
 'pediatrics': 'paediatrics',
 'pedophile': 'paedophile',
 'pedophiles': 'paedophiles',
 'pedophilia': 'paedophilia',
 'penalize': 'penalise',
 'penalized': 'penalised',
 'penalizes': 'penalises',
 'penalizing': 'penalising',
 'penciled': 'pencilled',
 'penciling': 'pencilling',
 'personalize': 'personalise',
 'personalized': 'personalised',
 'personalizes': 'personalises',
 'personalizing': 'personalising',
 'pharmacopeia': 'pharmacopoeia',
 'pharmacopeias': 'pharmacopoeias',
 'philosophize': 'philosophise',
 'philosophized': 'philosophised',
 'philosophizes': 'philosophises',
 'philosophizing': 'philosophising',
 'phony ': 'phoney ',
 'pizzazz': 'pzazz',
 'plagiarize': 'plagiarise',
 'plagiarized': 'plagiarised',
 'plagiarizes': 'plagiarises',
 'plagiarizing': 'plagiarising',
 'plow': 'plough',
 'plowed': 'ploughed',
 'plowing': 'ploughing',
 'plowman': 'ploughman',
 'plowmen': 'ploughmen',
 'plows': 'ploughs',
 'plowshare': 'ploughshare',
 'plowshares': 'ploughshares',
 'polarization': 'polarisation',
 'polarize': 'polarise',
 'polarized': 'polarised',
 'polarizes': 'polarises',
 'polarizing': 'polarising',
 'politicization': 'politicisation',
 'politicize': 'politicise',
 'politicized': 'politicised',
 'politicizes': 'politicises',
 'politicizing': 'politicising',
 'popularization': 'popularisation',
 'popularize': 'popularise',
 'popularized': 'popularised',
 'popularizes': 'popularises',
 'popularizing': 'popularising',
 'pouf': 'pouffe',
 'poufs': 'pouffes',
 'practice': 'practise',
 'practiced': 'practised',
 'practices': 'practises',
 'practicing ': 'practising ',
 'presidium': 'praesidium',
 'presidiums ': 'praesidiums ',
 'pressurization': 'pressurisation',
 'pressurize': 'pressurise',
 'pressurized': 'pressurised',
 'pressurizes': 'pressurises',
 'pressurizing': 'pressurising',
 'pretense': 'pretence',
 'pretenses': 'pretences',
 'primeval': 'primaeval',
 'prioritization': 'prioritisation',
 'prioritize': 'prioritise',
 'prioritized': 'prioritised',
 'prioritizes': 'prioritises',
 'prioritizing': 'prioritising',
 'privatization': 'privatisation',
 'privatizations': 'privatisations',
 'privatize': 'privatise',
 'privatized': 'privatised',
 'privatizes': 'privatises',
 'privatizing': 'privatising',
 'professionalization': 'professionalisation',
 'professionalize': 'professionalise',
 'professionalized': 'professionalised',
 'professionalizes': 'professionalises',
 'professionalizing': 'professionalising',
 'program': 'programme',
 'programs': 'programmes',
 'prolog': 'prologue',
 'prologs': 'prologues',
 'propagandize': 'propagandise',
 'propagandized': 'propagandised',
 'propagandizes': 'propagandises',
 'propagandizing': 'propagandising',
 'proselytize': 'proselytise',
 'proselytized': 'proselytised',
 'proselytizer': 'proselytiser',
 'proselytizers': 'proselytisers',
 'proselytizes': 'proselytises',
 'proselytizing': 'proselytising',
 'psychoanalyze': 'psychoanalyse',
 'psychoanalyzed': 'psychoanalysed',
 'psychoanalyzes': 'psychoanalyses',
 'psychoanalyzing': 'psychoanalysing',
 'publicize': 'publicise',
 'publicized': 'publicised',
 'publicizes': 'publicises',
 'publicizing': 'publicising',
 'pulverization': 'pulverisation',
 'pulverize': 'pulverise',
 'pulverized': 'pulverised',
 'pulverizes': 'pulverises',
 'pulverizing': 'pulverising',
 'pummel': 'pummelled',
 'pummeled': 'pummelling',
 'quarreled': 'quarrelled',
 'quarreling': 'quarrelling',
 'radicalize': 'radicalise',
 'radicalized': 'radicalised',
 'radicalizes': 'radicalises',
 'radicalizing': 'radicalising',
 'rancor': 'rancour',
 'randomize': 'randomise',
 'randomized': 'randomised',
 'randomizes': 'randomises',
 'randomizing': 'randomising',
 'rationalization': 'rationalisation',
 'rationalizations': 'rationalisations',
 'rationalize': 'rationalise',
 'rationalized': 'rationalised',
 'rationalizes': 'rationalises',
 'rationalizing': 'rationalising',
 'raveled': 'ravelled',
 'raveling': 'ravelling',
 'realizable': 'realisable',
 'realization': 'realisation',
 'realizations': 'realisations',
 'realize': 'realise',
 'realized': 'realised',
 'realizes': 'realises',
 'realizing': 'realising',
 'recognizable': 'recognisable',
 'recognizably': 'recognisably',
 'recognizance': 'recognisance',
 'recognize': 'recognise',
 'recognized': 'recognised',
 'recognizes': 'recognises',
 'recognizing': 'recognising',
 'reconnoiter': 'reconnoitre',
 'reconnoitered': 'reconnoitred',
 'reconnoitering': 'reconnoitring',
 'reconnoiters': 'reconnoitres',
 'refueled': 'refuelled',
 'refueling': 'refuelling',
 'regularization': 'regularisation',
 'regularize': 'regularise',
 'regularized': 'regularised',
 'regularizes': 'regularises',
 'regularizing': 'regularising',
 'remodeled': 'remodelled',
 'remodeling': 'remodelling',
 'remold': 'remould',
 'remolded': 'remoulded',
 'remolding': 'remoulding',
 'remolds': 'remoulds',
 'reorganization': 'reorganisation',
 'reorganizations': 'reorganisations',
 'reorganize': 'reorganise',
 'reorganized': 'reorganised',
 'reorganizes': 'reorganises',
 'reorganizing': 'reorganising',
 'reveled': 'revelled',
 'reveler': 'reveller',
 'revelers': 'revellers',
 'reveling': 'revelling',
 'revitalize': 'revitalise',
 'revitalized': 'revitalised',
 'revitalizes': 'revitalises',
 'revitalizing': 'revitalising',
 'revolutionize': 'revolutionise',
 'revolutionized': 'revolutionised',
 'revolutionizes': 'revolutionises',
 'revolutionizing': 'revolutionising',
 'rhapsodize': 'rhapsodise',
 'rhapsodized': 'rhapsodised',
 'rhapsodizes': 'rhapsodises',
 'rhapsodizing': 'rhapsodising',
 'rigor': 'rigour',
 'rigors': 'rigours',
 'ritualized': 'ritualised',
 'rivaled': 'rivalled',
 'rivaling': 'rivalling',
 'romanticize': 'romanticise',
 'romanticized': 'romanticised',
 'romanticizes': 'romanticises',
 'romanticizing': 'romanticising',
 'rumor': 'rumour',
 'rumored': 'rumoured',
 'rumors': 'rumours',
 'saber': 'sabre',
 'sabers': 'sabres',
 'saltpeter': 'saltpetre',
 'sanitize': 'sanitise',
 'sanitized': 'sanitised',
 'sanitizes': 'sanitises',
 'sanitizing': 'sanitising',
 'satirize': 'satirise',
 'satirized': 'satirised',
 'satirizes': 'satirises',
 'satirizing': 'satirising',
 'savior': 'saviour',
 'saviors': 'saviours',
 'savor': 'savour',
 'savored': 'savoured',
 'savories': 'savouries',
 'savoring': 'savouring',
 'savors': 'savours',
 'savory': 'savoury',
 'scandalize': 'scandalise',
 'scandalized': 'scandalised',
 'scandalizes': 'scandalises',
 'scandalizing': 'scandalising',
 'scepter': 'sceptre',
 'scepters': 'sceptres',
 'scrutinize': 'scrutinise',
 'scrutinized': 'scrutinised',
 'scrutinizes': 'scrutinises',
 'scrutinizing': 'scrutinising',
 'secularization': 'secularisation',
 'secularize': 'secularise',
 'secularized': 'secularised',
 'secularizes': 'secularises',
 'secularizing': 'secularising',
 'sensationalize': 'sensationalise',
 'sensationalized': 'sensationalised',
 'sensationalizes': 'sensationalises',
 'sensationalizing': 'sensationalising',
 'sensitize': 'sensitise',
 'sensitized': 'sensitised',
 'sensitizes': 'sensitises',
 'sensitizing': 'sensitising',
 'sentimentalize': 'sentimentalise',
 'sentimentalized': 'sentimentalised',
 'sentimentalizes': 'sentimentalises',
 'sentimentalizing': 'sentimentalising',
 'sepulcher': 'sepulchre',
 'sepulchers ': 'sepulchres',
 'serialization': 'serialisation',
 'serializations': 'serialisations',
 'serialize': 'serialise',
 'serialized': 'serialised',
 'serializes': 'serialises',
 'serializing': 'serialising',
 'sermonize': 'sermonise',
 'sermonized': 'sermonised',
 'sermonizes': 'sermonises',
 'sermonizing': 'sermonising',
 'sheik ': 'sheikh ',
 'shoveled': 'shovelled',
 'shoveling': 'shovelling',
 'shriveled': 'shrivelled',
 'shriveling': 'shrivelling',
 'signaled': 'signalled',
 'signaling': 'signalling',
 'signalize': 'signalise',
 'signalized': 'signalised',
 'signalizes': 'signalises',
 'signalizing': 'signalising',
 'siphon': 'syphon',
 'siphoned': 'syphoned',
 'siphoning': 'syphoning',
 'siphons': 'syphons',
 'skeptic': 'sceptic',
 'skeptical': 'sceptical',
 'skeptically': 'sceptically',
 'skepticism': 'scepticism',
 'skeptics': 'sceptics',
 'smolder': 'smoulder',
 'smoldered': 'smouldered',
 'smoldering': 'smouldering',
 'smolders': 'smoulders',
 'sniveled': 'snivelled',
 'sniveling': 'snivelling',
 'snorkeled': 'snorkelled',
 'snorkeling': 'snorkelling',
 'snowplow': 'snowploughs',
 'socialization': 'socialisation',
 'socialize': 'socialise',
 'socialized': 'socialised',
 'socializes': 'socialises',
 'socializing': 'socialising',
 'sodomize': 'sodomise',
 'sodomized': 'sodomised',
 'sodomizes': 'sodomises','sodomizing': 'sodomising','solemnize': 'solemnise','solemnized': 'solemnised',
 'solemnizes': 'solemnises','solemnizing': 'solemnising',
 'somber': 'sombre','specialization': 'specialisation','specializations': 'specialisations',
 'specialize': 'specialise','specialized': 'specialised','specializes': 'specialises','specializing': 'specialising',
 'specter': 'spectre','specters': 'spectres','spiraled': 'spiralled',
 'spiraling': 'spiralling','splendor': 'splendour','splendors': 'splendours','squirreled': 'squirrelled','squirreling': 'squirrelling',
 'stabilization': 'stabilisation','stabilize': 'stabilise','stabilized': 'stabilised','stabilizer': 'stabiliser',
 'stabilizers': 'stabilisers','stabilizes': 'stabilises','stabilizing': 'stabilising','standardization': 'standardisation',
 'standardize': 'standardise','standardized': 'standardised','standardizes': 'standardises','standardizing': 'standardising',
 'stenciled': 'stencilled','stenciling': 'stencilling','sterilization': 'sterilisation','sterilizations': 'sterilisations',
 'sterilize': 'sterilise','sterilized': 'sterilised','sterilizer': 'steriliser','sterilizers': 'sterilisers',
 'sterilizes': 'sterilises','sterilizing': 'sterilising','stigmatization': 'stigmatisation','stigmatize': 'stigmatise',
 'stigmatized': 'stigmatised','stigmatizes': 'stigmatises','stigmatizing': 'stigmatising','stories': 'storeys',
 'story': 'storey','subsidization': 'subsidisation','subsidize': 'subsidise','subsidized': 'subsidised',
 'subsidizer': 'subsidiser','subsidizers': 'subsidisers','subsidizes': 'subsidises','subsidizing': 'subsidising',
 'succor': 'succour','succored': 'succoured','succoring': 'succouring','succors': 'succours','sulfate': 'sulphate',
 'sulfates': 'sulphates','sulfide': 'sulphide','sulfides': 'sulphides','sulfur': 'sulphur','sulfurous': 'sulphurous'}

In [106]:
#Extra
df_mbti['Tokens'] = hero.tokenize(df_mbti['Text'])
SFW_13['clean_docs'] = SFW_13['prof + knowlg + ability'].apply(lambda x: x.replace('\n', '. '))

C:\Users\valli\anaconda3\lib\site-packages\texthero\preprocessing.py:598: FutureWarning: The default value of regex will change from True to False in a future version.
  return s.str.replace(pattern, r"\2 \3 \4 \5").str.split()
